In [1]:
# set working directory to same place ass app.py to import programs the same way as the app
import os
current_directory = os.getcwd()
if 'notebooks' in current_directory:
    parent_directory = os.path.abspath(os.path.join(current_directory, os.pardir))
    os.chdir(parent_directory)
os.getcwd()

'/Users/elisealstad/code/mybook-dashboard'

In [2]:
import pandas as pd 
import json
import numpy as np
pd.set_option('display.max_columns',100)


# Importing the data
- importing topics in json file
- import book df 

In [3]:
# mybooks = pd.read_csv('assets/goodreads_library_export.csv')

In [4]:
# mybooks.head(25).to_csv('assets/goodreads_library_export(1).csv')

In [5]:
# with open('assets/my_topics.json') as file:
#     json_data = json.load(file)

# # # Convert JSON data to a dictionary
# my_read_topics = dict(my_read_topics)

# Import good reads book from pickle

In [6]:
mybooks = pd.read_pickle('assets/my_books.pkl')

# # filer only books I have read
myreads = mybooks.query('Exclusive_Shelf == "read"').copy()
to_read = mybooks.query("Exclusive_Shelf == 'to-read'").copy()

# Fill missing date read with date added
myreads['Date_Read'] = myreads['Date_Read'].fillna(myreads['Date_Added'])
myreads = myreads.sort_values(by='Date_Read')

In [7]:
myreads.head()


,Book_Id,Title,Author,Author_l-f,Additional_Authors,ISBN_Goodreads,ISBN13,My_Rating,Average_Rating_Goodreads,Publisher,Binding,Number_of_Pages,Year_Published,Original_Publication_Year,Date_Read,Date_Added,Bookshelves,Bookshelves_with_positions,Exclusive_Shelf,My_Review,Spoiler,Private_Notes,Read_Count,Owned_Copies,Author(s),Publish_Date,Description,ISBN_GoogleBooks,Page_Count,Categories,Average_Rating_GoogleBooks,Rating_Count,Language,Page_Cat,Year,Quarter,Year_Quarter
363,774310,Animal Farm,George Orwell,"Orwell, George",NaN,"=""""","=""""",5,3.98,Penguin Books,Paperback,120.0,2000,1945,2020-06-19,2020/06/19,NaN,NaN,read,NaN,NaN,NaN,1,0,George Orwell,2011,Mr Jones of Manor Farm is so lazy and drunken ...,0143416316,94.0,NaN,4.0,1825.0,en,100-249,2020.0,2.0,2020-Q2
362,947755,Monkey Hunting: A Novel (Ballantine Reader's C...,Cristina García,"García, Cristina",NaN,"=""0345466101""","=""9780345466105""",4,3.65,Ballantine Books,Paperback,288.0,2004,2003,2020-06-19,2020/06/19,NaN,NaN,read,NaN,NaN,NaN,1,0,Cristina García,2007-12-18,"In this deeply stirring novel, acclaimed autho...",9780307416100,288.0,Fiction,2.5,2.0,en,250-349,2020.0,2.0,2020-Q2
272,2657,To Kill a Mockingbird,Harper Lee,"Lee, Harper",NaN,"=""""","=""""",4,4.26,Harper Perennial Modern Classics,Paperback,323.0,2006,1960,2020-06-19,2020/06/19,NaN,NaN,read,NaN,NaN,NaN,1,0,Harper Lee,2004,Theatre program.,9780099466734,323.0,Fiction,NaN,NaN,en,250-349,2020.0,2.0,2020-Q2
273,19063,The Book Thief,Markus Zusak,"Zusak, Markus",NaN,"=""""","=""""",4,4.39,Alfred A. Knopf,Hardcover,592.0,2006,2005,2020-06-19,2020/06/19,NaN,NaN,read,NaN,NaN,NaN,1,0,Markus Zusak,2007-09-11,"DON'T MISS BRIDGE OF CLAY, MARKUS ZUSAK'S FIRS...",PSU:000061286726,582.0,Young Adult Fiction,4.5,2494.0,en,450-599,2020.0,2.0,2020-Q2
275,929,Memoirs of a Geisha,Arthur Golden,"Golden, Arthur",NaN,"=""1400096898""","=""9781400096893""",4,4.14,Vintage Books USA,Mass Market Paperback,503.0,2005,1997,2020-06-19,2020/06/19,NaN,NaN,read,NaN,NaN,NaN,1,0,Arthur Golden,1999-01-10,"A literary sensation and runaway bestseller, t...",0679781587,452.0,Fiction,4.0,1334.0,en,450-599,2020.0,2.0,2020-Q2


In [28]:
## 
import plotly.express as px

# books read year
def viz_read(df):
    
    df = df.dropna(subset=['Year'])
    
    # Count the number of books read in each year and quarter
    year_quarter_counts = df.groupby('Year',  observed=True).size().reset_index(name='Books Read')
    
    # Create the line chart
    fig = px.line(year_quarter_counts, x='Year', y='Books Read', markers=True, template = "plotly_white")
    fig['data'][0]['line']['color']='#A777F1'
    # Customize the plot layout
    fig.update_layout(
        title='Number of books read - Timeline',
        xaxis=dict(title='Year', dtick=1),
        yaxis=dict(title='Number of Books Read', range=[year_quarter_counts['Books Read'].min()-20, year_quarter_counts['Books Read'].max()+20]),
        
        showlegend=False

    )
    return fig

In [29]:
viz_read(myreads)


## Scatter plot rating count and ratings

In [4]:
# Create scatter plot
import plotly.express as px
import plotly.graph_objects as go
df = myreads 
df['My_Rating'] = df['My_Rating'].replace(0, np.nan)
df = df.dropna(subset=['Rating_Count', 'Average_Rating_Goodreads'])
# Melt the dataframe to transform it into the desired format
melted_df = pd.melt(df[['Rating_Count', 'Average_Rating_Goodreads', 'My_Rating']], id_vars=['Rating_Count'], value_vars=['My_Rating', 'Average_Rating_Goodreads'],
                    var_name='Rating_Type', value_name='Rating')


# fig = px.scatter(melted_df, x='Rating_Count', y='Rating',color='Rating_Type', trendline="ols", title="Do popularity correlate with rating?<br>")

color_scale = px.colors.qualitative.Set2[2:8]  # Choose your desired color scale

# Create a scatter plot using Plotly Express with white template and custom colors
fig = px.scatter(
    melted_df, x='Rating_Count', y='Rating',color='Rating_Type', trendline="ols", title="Do popularity correlate with rating?<br>",
    template="plotly_white",  # Set the template to white
    color_discrete_sequence=color_scale,  # Set the color scale
    labels={"Rating_Type": "Rating", 'Average_Rating_Goodreads': "Average Rating on Goodreads"}
)
# Customize the legend
fig.update_layout(
    legend=dict(
        title='Rating Type',  # Set legend title
        orientation='h',  # Set legend orientation (horizontal)
        x=0.5,  # Adjust x position of the legend
        y=1.2,  # Adjust y position of the legend
    ),
    
    xaxis=dict(
        title='Number of times rates in Google Books',  # Set X-axis label
    ),
    
    yaxis=dict(
        title='Custom Y-Axis Label',  # Set Y-axis label
    )
)

# Show the plot
fig

### Import topics dict from json file

In [8]:
import json

# Load JSON data from a file
with open('assets/my_topics.json') as file:
    json_data = json.load(file)

# Convert JSON data to a dictionary
my_topics = dict(json_data)

In [9]:
# creating dictionaries for read and want-to-read books
my_read_topics = {k: v for k, v in my_topics.items() if k in myreads.Title.to_list()}
my_want_titles = list(set(list(my_topics.keys())) - set(myreads.Title.to_list()))
my_want_topics = {k: v for k, v in my_topics.items() if k in my_want_titles}

In [10]:
import plotly.express as px
from collections import Counter

word_counts_dict = Counter()

# Count the occurrences of each word in the values of the original dictionary
for values in my_read_topics.values():
    for value in values:
        for word in value.replace(', ', ',').split(','):
            word_counts_dict[word.lower()] += 1

df_word_counts = pd.DataFrame(list(word_counts_dict.items()), columns=['Word', 'Count'])
df_word_counts = df_word_counts.sort_values(by='Count', ascending=False).reset_index().head(45)

fig = px.treemap(
    df_word_counts,
    path=['Word'],
    values='Count',
    custom_data=['Count'] )

# Update the layout of the tree map
fig.update_layout(
    title='Most common topics'
)
fig.update_traces(
    hovertemplate='<b>%{label}</b><br>Count: %{customdata[0]}'
)
# Show the plot
fig.show()

# Topics visualization 

In [11]:
from apps.viz import tree_topics
fig = tree_topics(my_read_topics)
fig.show()

# vizualising publication year

In [12]:
from apps.viz import viz_pub_year
fig = viz_pub_year(myreads)
fig.show()

# Vizualising timeline of books read. 

In [13]:
from apps.viz import viz_year_read
viz_year_read(myreads).show()

/Users/elisealstad/Desktop/Code/mybook-dashboard/apps/viz.py:116: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



# Number of pages and read count

Visualising the most read page count of books

In [14]:
from apps.viz import visualize_page_categories

# Assuming you have a DataFrame 'myreads' with 'Page_Cat' column

visualize_page_categories(myreads, 'My_Rating', 'How do I rate my books?<br><span style="font-size: 8px;">Number of books per Ratings category</span>', 'Goodreads rating')



ImportError: cannot import name 'visualize_page_categories' from 'apps.viz' (/Users/elisealstad/Desktop/Code/mybook-dashboard/apps/viz.py)

In [ ]:
# Assuming you have a DataFrame 'myreads' with 'Page_Cat' column

visualize_page_categories(myreads, 'Page_Cat', 'How long are the books I read?<br><span style="font-size: 8px;">Number of books per Page Count Category</span>', 'Page Count Category')

# Vizualise top categories and languages
I am using a pie chart here because I excpect few values in each variable and a large discrepancy.

In [ ]:
from apps.viz import viz_top_values

In [ ]:
viz_top_values(mybooks['Language'], top_n=5)

In [ ]:
viz_top_values(mybooks['Categories'], top_n=10)

## Rating visualised 

In [32]:
def book_ratings_top(data, title_txt):
    # Define custom colors
    my_rating_color = 'rgb(180,151,231)'
    google_books_color = '#34A853'
    goodreads_color = '#e9e5cd'

    # Filter the data where My_Rating > 0 since this would include non-rated books
    filtered_data = data[data['My_Rating'] > 0]

    # Sort the filtered data by your own rating in ascending order for bottom-rated books
    filtered_data = filtered_data.sort_values(['My_Rating', 'Date_Read'], ascending=True)

    # Select the top 10 books based on your own rating
    top_books = filtered_data.tail(15)

    # Create the figure object
    fig = px.scatter(template = "plotly_white")

    # Add trace for Average_Rating_GoogleBooks as dots with a different color and shape
    fig.add_trace(go.Scatter(
        x=top_books['Average_Rating_GoogleBooks'],
        y=top_books['Title'],
        mode='markers',
        name=f"Average Rating Google Books",
        marker=dict(color=google_books_color, symbol='square', size=15),
    ))

    # Add trace for Average_Rating_Goodreads as dots with a different color and shape
    fig.add_trace(go.Scatter(
        x=top_books['Average_Rating_Goodreads'],
        y=top_books['Title'],
        mode='markers',
        name='Average Rating Goodreads',
        marker=dict(color=goodreads_color, symbol='diamond', size=15),
    ))

    # Add trace for My Rating as dots
    fig.add_trace(go.Scatter(
        x=top_books['My_Rating'],
        y=top_books['Title'],
        mode='markers',
        name='My Rating',
        marker=dict(color=my_rating_color, symbol='circle', size=15),
    ))

    # Update the layout
    fig.update_layout(
        title= f'{title_txt}<br><span style="font-size: 11px;">*Showing only 15 latest read books</span>',
        # font=dict(size=12),
        yaxis=dict(title='Title', side='top', showticklabels=True),
        xaxis=dict(
            title='Rating',
            tickmode='array',
            tickvals=[1, 2, 3, 4, 5],
            range=[0.5, 5.5],  # Set the X-axis range from 0 to 5, 
            showgrid=True),
        legend=dict(
            title='Rating',  # Set legend title
            orientation='h',  # Set legend orientation (horizontal)
            x=0.01,  # Adjust x position of the legend
            y=1.2,  # Adjust y position of the legend
        )
        )
    return fig


def book_ratings_bottom(data, title_txt):
    # Define custom colors
    my_rating_color = 'rgb(180,151,231)'
    google_books_color = '#34A853'
    goodreads_color = '#e9e5cd'

    # Filter the data where My_Rating > 0 since this would include non-rated books
    filtered_data = data[data['My_Rating'] > 0]

    # Sort the filtered data by your own rating in ascending order for bottom-rated books
    filtered_data = filtered_data.sort_values(['My_Rating', 'Date_Read'], ascending=True)

    # Select the top 10 books based on your own rating
    top_books = filtered_data.head(15)

    # Create the figure object
    fig = px.scatter(template = "plotly_white")

    # Add trace for My Rating as dots
    fig.add_trace(go.Scatter(
        x=top_books['My_Rating'],
        y=top_books['Title'],
        mode='markers',
        name='My Rating',
        marker=dict(color=my_rating_color, symbol='circle', size=15),
    ))

    # Add trace for Average_Rating_GoogleBooks as dots with a different color and shape
    fig.add_trace(go.Scatter(
        x=top_books['Average_Rating_GoogleBooks'],
        y=top_books['Title'],
        mode='markers',
        name=f"Average Rating Google Books",
        marker=dict(color=google_books_color, symbol='square', size=15),
    ))

    # Add trace for Average_Rating_Goodreads as dots with a different color and shape
    fig.add_trace(go.Scatter(
        x=top_books['Average_Rating_Goodreads'],
        y=top_books['Title'],
        mode='markers',
        name='Average Rating Goodreads',
        marker=dict(color=goodreads_color, symbol='diamond', size=15),
    ))

    # Update the layout
    fig.update_layout(
        title= f'{title_txt}<br><span style="font-size: 11px;">*Showing only 15 latest read books</span>',
        # font=dict(size=12),
        yaxis=dict(title='Title', side='top', showticklabels=True),
        xaxis=dict(
            title='Rating',
            tickmode='array',
            tickvals=[1, 2, 3, 4, 5],
            range=[0.5, 5.5],  # Set the X-axis range from 0 to 5, 
            showgrid=True),
        legend=dict(
            title='Rating',  # Set legend title
            orientation='h',  # Adjust x position of the legend
            y=-0.15  # Adjust y position of the legend
        )
        )
    return fig




In [1]:
book_ratings_top(myreads, 'Top Rated Books')
book_ratings_bottom(myreads, 'Lowest Rated Books')

NameError: name 'book_ratings_top' is not defined

## Top authors 

In [11]:
import plotly.express as px

In [21]:
tbl_authors = myreads.Author.value_counts().head(20).reset_index().sort_values(by=['count'], ascending = True)


In [25]:
def author_count_fig(myreads):
        
        tbl_authors = myreads.Author.value_counts().head(20).reset_index().sort_values(by=['count'], ascending = True)

        fig_author = px.bar(tbl_authors, x = 'count', y = 'Author', orientation = 'h', labels = {'count' : 'Number of books read by author'})

        fig_author.update_layout(title = 'My most read authors',
                template='plotly_white',
                plot_bgcolor='white',
                paper_bgcolor='white')

        fig_author.update_traces(marker_color='#C1E1C1', width=0.4)
        return fig_author

author_count_fig(myreads)

In [26]:
def author_rating_fig(myreads): 
    
    ls_authors = myreads.Author.value_counts().head(20).reset_index().sort_values(by=['count'], ascending = True)['Author']

    myreads['My_Rating'] = myreads['My_Rating'].replace(0, np.nan)

    # Calculate the mean 'My_Rating' grouped by 'Author'
    myreads['mean_rating_by_author'] = myreads.groupby('Author',  observed=True)['My_Rating'].transform('mean').copy()

    myreads['My_Rating'] = np.where((myreads['My_Rating'] == np.nan) | myreads['My_Rating'].isnull(), myreads['mean_rating_by_author'], myreads['My_Rating'])


    tbl_rating_authors = (
        myreads.query('Author in @ls_authors')
        .groupby('Author')['My_Rating'].mean().reset_index()
        ).set_index('Author').reindex(index = ls_authors).reset_index()


    fig_author_rating = px.bar(tbl_rating_authors, x = 'My_Rating', y = 'Author', orientation = 'h', labels = {'My_Rating' : 'My average rating of author'})

    fig_author_rating.update_layout(title = 'My average rating of my most read authors',
            template='plotly_white',
            plot_bgcolor='white',
            paper_bgcolor='white')

    fig_author_rating.update_traces(marker_color='#FFD580', width=0.4)
    
    return fig_author_rating

author_rating_fig(myreads)

# This year in books

In [ ]:
from datetime import datetime
datetime.today().year

2023

In [ ]:
# Filter on books read this year using datetime to find todays year
from datetime import datetime
today_year = datetime.today().year
print(f"This year you have read {len(myreads.query('Year == @today_year'))} books. Totaling {f'{(myreads.Number_of_Pages.sum().astype(int)):,}'} pages read.")

This year you have read 51 books. Totaling 62,094 pages read.


# Word cloud from Goodreads book Description

In [ ]:

# Three viz of count of words in description
import numpy as np
import plotly.express as px
import re
from collections import Counter

def desc_tree(Description):
    from collections import Counter
    stopwords_dict = {'like', 'author', 'S', 'will','new','york','time','book','novel', 'read', 'day', 'make','year', 'one', 'times', 'times, of', 's', 'award','author','new','york','selling','story','t','1','og', 'call', 'upon', 'still', 'nevertheless', 'down', 'every', 'forty', '‘re', 'always', 'whole', 'side', "n't", 'now', 'however', 'an', 'show', 'least', 'give', 'below', 'did', 'sometimes', 'which', "'s", 'nowhere', 'per', 'hereupon', 'yours', 'she', 'moreover', 'eight', 'somewhere', 'within', 'whereby', 'few', 'has', 'so', 'have', 'for', 'noone', 'top', 'were', 'those', 'thence', 'eleven', 'after', 'no', '’ll', 'others', 'ourselves', 'themselves', 'though', 'that', 'nor', 'just', '’s', 'before', 'had', 'toward', 'another', 'should', 'herself', 'and', 'these', 'such', 'elsewhere', 'further', 'next', 'indeed', 'bottom', 'anyone', 'his', 'each', 'then', 'both', 'became', 'third', 'whom', '‘ve', 'mine', 'take', 'many', 'anywhere', 'to', 'well', 'thereafter', 'besides', 'almost', 'front', 'fifteen', 'towards', 'none', 'be', 'herein', 'two', 'using', 'whatever', 'please', 'perhaps', 'full', 'ca', 'we', 'latterly', 'here', 'therefore', 'us', 'how', 'was', 'made', 'the', 'or', 'may', '’re', 'namely', "'ve", 'anyway', 'amongst', 'used', 'ever', 'of', 'there', 'than', 'why', 'really', 'whither', 'in', 'only', 'wherein', 'last', 'under', 'own', 'therein', 'go', 'seems', '‘m', 'wherever', 'either', 'someone', 'up', 'doing', 'on', 'rather', 'ours', 'again', 'same', 'over', '‘s', 'latter', 'during', 'done', "'re", 'put', "'m", 'much', 'neither', 'among', 'seemed', 'into', 'once', 'my', 'otherwise', 'part', 'everywhere', 'never', 'myself', 'must', 'will', 'am', 'can', 'else', 'although', 'as', 'beyond', 'are', 'too', 'becomes', 'does', 'a', 'everyone', 'but', 'some', 'regarding', '‘ll', 'against', 'throughout', 'yourselves', 'him', "'d", 'it', 'himself', 'whether', 'move', '’m', 'hereafter', 're', 'while', 'whoever', 'your', 'first', 'amount', 'twelve', 'serious', 'other', 'any', 'off', 'seeming', 'four', 'itself', 'nothing', 'beforehand', 'make', 'out', 'very', 'already', 'various', 'until', 'hers', 'they', 'not', 'them', 'where', 'would', 'since', 'everything', 'at', 'together', 'yet', 'more', 'six', 'back', 'with', 'thereupon', 'becoming', 'around', 'due', 'keep', 'somehow', 'n‘t', 'across', 'all', 'when', 'i', 'empty', 'nine', 'five', 'get', 'see', 'been', 'name', 'between', 'hence', 'ten', 'several', 'from', 'whereupon', 'through', 'hereby', "'ll", 'alone', 'something', 'formerly', 'without', 'above', 'onto', 'except', 'enough', 'become', 'behind', '’d', 'its', 'most', 'n’t', 'might', 'whereas', 'anything', 'if', 'her', 'via', 'fifty', 'is', 'thereby', 'twenty', 'often', 'whereafter', 'their', 'also', 'anyhow', 'cannot', 'our', 'could', 'because', 'who', 'beside', 'by', 'whence', 'being', 'meanwhile', 'this', 'afterwards', 'whenever', 'mostly', 'what', 'one', 'nobody', 'seem', 'less', 'do', '‘d', 'say', 'thus', 'unless', 'along', 'yourself', 'former', 'thru', 'he', 'hundred', 'three', 'sixty', 'me', 'sometime', 'whose', 'you', 'quite', '’ve', 'about', 'even'}
        
    descriptions_all = ' '.join(Description.dropna()).lower()

    descriptions_all = re.findall(r'\b\w+\b', descriptions_all.lower())

    # Remove stop words from the list of words
    filtered_words = [word for word in descriptions_all if word not in stopwords_dict]

    word_counts = Counter(filtered_words)

    # Create a DataFrame from the word counts
    word_counts_df = pd.DataFrame(word_counts.items(), columns=['Word', 'Count'])
    # Create a tree map using Plotly Express
    fig = px.treemap(word_counts_df.sort_values(by='Count', ascending=False).head(55), path=['Word'], values='Count', custom_data=['Count'] )

    # Update the layout of the tree map
    fig.update_layout(
        title='Most common words found in book descriptions'
    )
    fig.update_traces(
        hovertemplate='<b>%{label}</b><br>Count: %{customdata[0]}'
    )

    return fig

desc_tree(mybooks['Description'])

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/elisealstad/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Genre visualizaiton

In [2]:
# import data and clean

import pandas as pd

nmyreads= pd.read_pickle('../assets/my_books_genres.pickle')
# genre table 
from ast import literal_eval
nmyreads['genres'] = nmyreads['genres'].apply(literal_eval)
nallgenredf = nmyreads.query('genres != "[]"').copy().explode('genres')


def make_genre_tbl(df_genres):
    import numpy as np 
    tbl_genre = (
        pd.DataFrame(
            np.round(
                (
                df_genres.query('Read_Count > 0 & My_Rating > 0')
                .groupby('genres').aggregate({'genres':'count', 'My_Rating':'mean'})
                )
            ,2)
        # remove fiction since there is too many and only count above 2
        ).query('genres > 5 ')
        # .drop(['Fiction'])
    )
    return tbl_genre



tbl_genre = make_genre_tbl(nallgenredf)

In [11]:
tbl_genre.My_Rating.max()

4.09

In [25]:
import plotly.express as px
import plotly.graph_objects as go



 

lolli_fig(tbl_genre)

In [9]:
import plotly.express as px

def spider_fig(tbl_genre):
    spider_fig = px.line_polar(tbl_genre, r=tbl_genre.My_Rating, theta=tbl_genre.index, line_close=True, range_r=[1, tbl_genre.My_Rating.max()+0.2]).update_traces(fill='tonext', fillcolor='rgba(167, 119, 241, 0.5)')
    spider_fig.update_layout(template='plotly_white')
    spider_fig.show()

# Assuming tbl_genre is your DataFrame containing the necessary data
spider_fig(tbl_genre)

In [342]:
import plotly.express as px

def stack_fig(allgenredf):
    
    tbl_percent = (
        pd.DataFrame(
            allgenredf.query('Read_Count > 0 & genres in @tbl_genre.index.tolist()')
            .groupby('Year_Quarter')['genres'].value_counts(normalize = True)
            )
        .reset_index()
        .query('Year_Quarter != "" and proportion > 0.01')
    )
    
    fig = px.area(tbl_percent, x='Year_Quarter',y='proportion', color='genres', line_group='genres',
                title='Do I go through periods were I read more of the same genres?<span style="font-size: 10px;"><br>Figure shows the proportion of books in each genre read in that year-quarter. A book can have multiple genres.</span>')
        
    fig.update_layout( 
        xaxis_title='Year quarter',
        yaxis_title='Proportion of books read with genre',
        legend_title="Genre",
        template="plotly_white")

    fig.show()
    
stack_fig(allgenredf)

/var/folders/nq/9vkk1x4j3791_ywhx015m2rh0000gn/T/ipykernel_55887/644924705.py:8: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [18]:
# Table with top authors , rating and count read
from datetime import datetime, timedelta
today = datetime.today()
# Subtract one year from today's date
one_year_ago = today - timedelta(days=365)

year_text = f"In the last 12 months you have read {len(myreads.query('Date_Read > @one_year_ago'))} books. Totaling {(myreads.query('Date_Read > @one_year_ago').Number_of_Pages.sum().astype(int))} pages read. Enjoy exploring your reading stats!"